# ✔	 - Import modules -

In [1]:
import os
import time
import numpy as np
import random
from matplotlib import pyplot as plt
from math import exp
import json

# ✔	 - Function definition -

# ✘ Corrected Input function

In [2]:
# Function returning the corrected Input
def Ic(Input=0, threshold=0):
    if Input - threshold <= 0:
        return 0
    else:
        return (Input - threshold)

# ✘ Activity at time t-Delta (delay) function

In [3]:
# function to get the index "-delay/dt" in the list representing m(t-Delta)
def mdelta(synapse_value, delay, t, dt,i):
    if t < delay:  # if the delay is the time is too small, we take m(t-delta)=0
        return 0
    else:  # if the time is above or equal to the delay, we can find the m(t-delta)
        return synapse_value[int(-delay / dt)][i]

# ✘ Plotting functions (Inputs & activities) and save data function

In [4]:
def plottingI(k):
    valueGpe=[0]*n_period
    valueFsi=[0]*n_period
    valueMsn=[0]*n_period
    for stc in structures:
        for t in range(1,n_period):
        
            valueGpe[t]=I['Gpe'][t][k]
            valueFsi[t]=I['Fsi'][t][k]
            valueMsn[t]=I['Msn'][t][k]
        
    fig = plt.figure(figsize=(9,5))
    ax=fig.add_subplot(111)
    ax.plot(time_value, valueGpe, label='Msn')
    ax.plot(time_value, valueFsi, label='Gpe')
    ax.plot(time_value, valueMsn, label='Fsi')
    ax.set_title('Input over time: Neuron {}'.format(k+1))
    ax.set_ylabel('Input')
    ax.set_xlabel('time')
    ax.grid()
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)

In [5]:
def plottingA(k):    # better define that function? with choice name etc
    valueGpeMsn=[0]*n_period
    valueFsiGpe=[0]*n_period
    valueMsnFsi=[0]*n_period
    for con in connections:
        for t in range(1,n_period):
        
            valueGpeMsn[t]=m['GpeMsn'][t][k]
            valueFsiGpe[t]=m['FsiGpe'][t][k]
            valueMsnFsi[t]=m['MsnFsi'][t][k]
        
    fig = plt.figure(figsize=(9,5))
    ax=fig.add_subplot(111)
    ax.plot(time_value, valueGpeMsn, label='Msn')
    ax.plot(time_value, valueFsiGpe, label='Gpe')
    ax.plot(time_value, valueMsnFsi, label='Fsi')
    ax.set_title('Input over time: Neuron {}'.format(k+1))
    ax.set_ylabel('Activity')
    ax.set_xlabel('time')
    ax.grid()
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0) 

In [6]:
def data_dir(dir="os.getcwd()"):
    directory = os.path.dirname(dir)
    if not os.path.exists('{}/{}'.format(os.getcwd(),'Data')):
        os.makedirs('{}/{}'.format(os.getcwd(),'Data'))
    if not os.path.exists('Data/{}'.format(timestr)):
        os.makedirs('Data/{}'.format(timestr))

# ✔	 - Initial parameters - 

In [7]:
# Population size (number of neuron)
n = 5

# Default Time beginning (second)
t = 0

# Default trial duration (second)
duration = 1.00   

# Default Time resolution (second)
dt = 0.0001  # 0.0001

# Period occuring 
n_period = int(duration // dt) + 1

# Initialization of the random generator (reproductibility)
np.random.seed(1)

In [8]:
time_value = np.arange(0, duration, dt) # creat the timescale, depending on the duration and step dt, for plotting

In [9]:
connections = ["GpeMsn", "FsiGpe", "MsnFsi"] #"MsnMsn", "FsiFsi", "GpeGpe"]
structures = ["Msn", "Fsi", "Gpe"]

In [10]:
                                                  # need IN VIVO Values + calculate them f(weight,I,m)
h = {  
    "Msn": -51.90,   # Mahon et al., 2006. RAT (In Vitro)
    "Fsi": -47.30,   # Koos & Teper, 1999. RAT (In Vitro)
    "Gpe": -55.10,   # Abdi, Mallet et al., 2015. RAT (In Vitro)
}

 # Weights (connextions pre->post ; notation: PostPre)
G = {
    "GpeMsn": 0.10,
    "FsiGpe": 0.10,
    "MsnFsi": 0.10,
    #"MsnMsn": 1.00,
    #"FsiFsi": 1.00,
    #"GpeGpe": 1.00
}

 # tau:
tau = {
    "GpeMsn": 0.0200,
    "FsiGpe": 0.0200,
    "MsnFsi": 0.0200, #0.0150 Koos & Tepper, 1999 : (in vitro - rat)
    # "MsnMsn": 1.00,
    #"FsiFsi": 1.00,
   # "GpeGpe": 1.00
}

 # Delays:
D = {
    "GpeMsn": 0.0070, # Jaeger & Kita, 2011 ; (in vivo-rat & monkey)
    "FsiGpe": 0.0010, # Jaeger & Kita, 2011 ; (in vivo-rat & monkey)
    "MsnFsi": 0.0057, # Koos & Tepper, 1999 : (in vitro - rat)
    #"MsnMsn": 1.00,
    #"FsiFsi": 1.00,
   # "GpeGpe": 1.00  # Jaeger & Kita, 2011 ; (in vivo-rat & monkey)
}


 # Noise level (sigma)
N = {
    "Msn": 0.00,   
    "Fsi": 0.00,   
    "Gpe": 0.00
}

In [11]:
# creation of array of n neurons to store m, dm and connectivity probability(0 or 1)
m, dm, J = dict(), dict(), dict()

for con in connections:   # create an array of 'nperiod' array(to store the value trought the time) with n zeroes
    m[con] = np.zeros((n_period, n))
    dm[con] = np.zeros((n_period, n))
    
    if con  == 'GpeMsn':         # create an array composed of 0 & 1 depending on the probability "Co"(max=1=100%)
        Co=1.000
    elif con == 'FsiGpe':
        Co=1.000
    elif con == 'MsnFsi':
        Co=1.000
        
    J[con] = np.random.choice(2, size=(n, n), p=[(1-Co),Co])

In [12]:
# inputs: pick noise value before iteration for calculation optimisation
I = dict()
for stc in structures: # create an array of 'nperiod' line & n column(to store value trought the time) with noise
    I[stc] = np.random.normal(0, 1, size=(n_period, n)) # Pick randomly a noise for each value of the array 
    
    # put dependency on sigma!

In [ ]:
input_mapping = {
    "Msn": "MsnFsi",
    "Gpe": "GpeMsn",
    "Fsi": "FsiGpe"
}

activities_mapping = {
    "GpeMsn": "Msn",
    "FsiGpe": "Gpe",
    "MsnFsi": "Fsi"

}

# ✔	 - Simulation Core -

In [ ]:
for t in range(1, n_period):    # Update of Inputs
    
    for stc in structures:
        for i in range(n):
            if stc == "Msn" or "Gpe" or "Fsi":
                con = input_mapping[stc]
                # I[stc][t][i] += - sum([G[con] * x * (mdelta(m[con][:t + 1], D[con], time_value[t], dt,i)) for x in J[con][i]])
                I[stc][t][i] += - sum(G[con]*(mdelta(m[con][:t + 1], D[con], time_value[t], dt,i))* J[con][i])
                print(G[con]*(mdelta(m[con][:t + 1], D[con], time_value[t], dt,i))* J[con][i])
            else:
                pass

    for con in connections:     # Update of activities
        for i in range(n):
            stc = activities_mapping[con]
            dm[con][t][i] = dt * (- m[con][t][i] + Ic(I[stc][t][i], h[stc])) / tau[con] #+ float(noise(1, N[stc]))
        if t < n_period-1:
            m[con][t+1]=m[con][t]+dm[con][t]
        else:
            pass

[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  

# ✔	 - Result Plots & Save data -

In [ ]:
timestr = time.strftime("%Y-%m-%d-%H:%M:%S") # give the date and time that will be used to tag simulation results
data_dir()  # creating the 'Data' directory on CWD and the directory (with date & time) to store the results

for stc in structures:  # save the Input array (npy format) for each neuron population in "structures"
    np.save('Data/{}/{}_Inputs'.format(timestr, stc), I[stc])
    
for con in connections: # save the Activity array (npy format) for each neuron population in "connections"
    np.save('Data/{}/{}_Activities'.format(timestr, con), m[con])

datax={'Weight':G,'Threshold':h,'Delay':D,'tau':tau,'Noise':N,  # make one dict with all parameters used 
       'Neuron_number':n,'time_trial':t,'duration':duration,'step':dt}

with open('Data/{}/0_Parameters.json'.format(timestr),'w') as f: # save the parameters used during the simulation
            json.dump(datax, f) 

In [ ]:
# Plotting INPUTS
for k in range(0,n):
    plottingI(k)
    plt.savefig('Data/{}/In{}.png'.format(timestr,k+1),dpi=300,bbox_inches='tight')
    plt.show()

In [ ]:
# Plotting ACTIVITIES
for k in range(0,n):
    plottingA(k)
    plt.savefig('Data/{}/Mn{}.png'.format(timestr,k+1),dpi=300,bbox_inches='tight')
    plt.show()

# ✔	 - Mean firing rate plotting - 

In [ ]:
MeaniGM=list()
MeaniMF=list()
MeaniFG=list()
for con in connections:
    for t in range(0,n_period):
        if con  == 'GpeMsn':
            MeaniGM.append(np.mean(m['GpeMsn'][t]))
        elif con == 'MsnFsi':
            MeaniMF.append(np.mean(m['MsnFsi'][t]))
        elif con == 'FsiGpe':
            MeaniFG.append(np.mean(m['FsiGpe'][t]))  

plt.plot(time_value,MeaniGM,label='Msn')
plt.plot(time_value,MeaniFG,label='Gpe')
plt.plot(time_value,MeaniMF,label='Fsi')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.grid()
plt.show()